In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

%matplotlib inline

In [ ]:
HEAD_PATH = '/kaggle/input/coughing-frames/'

In [ ]:
df = pd.read_csv(HEAD_PATH + 'frames_label.csv')
df = df.fillna(0)
df.head()

In [ ]:
SPLIT_INDEX = 13768 # set to video 18-20

train_df = df.iloc[0:SPLIT_INDEX]
test_df = df.iloc[SPLIT_INDEX:]

In [ ]:
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [ ]:
plt.figure(figsize=(10,4))

plt.subplot(1,2,1)
sns.countplot(train_df['label'])
plt.title('train_df')

plt.subplot(1,2,2)
sns.countplot(train_df['label'])
plt.title('test_df')

In [ ]:
from keras.preprocessing import image

def create_balance_df(train_df):
    not_cough_indices = train_df[train_df['label'] == 0].index
    random_indices = np.random.choice(not_cough_indices, train_df[train_df['label'] == 1]['label'].count(), replace=False)
    not_cough_sample = train_df.loc[random_indices]

    train_df = train_df[train_df['label'] == 1].append(not_cough_sample)
    return train_df

def get_image_array(file_list):
    image_list = []

    for file in file_list:
        file = HEAD_PATH + 'frames/' + file[2:]
        im = image.load_img(file, target_size=(224, 224, 3))
        im = image.img_to_array(im)
        im = im/255
        image_list.append(im)
        
    image_list = np.array(image_list)
    return image_list

In [ ]:
train_df = create_balance_df(train_df)
train_df = train_df.sample(frac=1).reset_index(drop=True)
sns.countplot(train_df['label'])

In [ ]:
train_X, train_y = train_df['file'], train_df['label']
test_X, test_y = test_df['file'], test_df['label']

print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

In [ ]:
train_X = get_image_array(train_X)
test_X = get_image_array(test_X)
train_X.shape

In [ ]:
from keras.applications.vgg16 import VGG16

base_model = VGG16(weights='imagenet', include_top=False)

In [ ]:
train_X = base_model.predict(train_X)
shape = train_X.shape[1] * train_X.shape[2] * train_X.shape[3]
train_X.shape

In [ ]:
train_X = train_X.reshape(train_X.shape[0], shape)
train_X = train_X/train_X.max()

train_X.shape

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

#defining the model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(shape,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
from keras.callbacks import ModelCheckpoint

mcp_save = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
# compiling the model
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(train_X, train_y, epochs=100, validation_split=0.2, callbacks=[mcp_save], batch_size=128)

In [ ]:
epochs = [i for i in range(len(model.history.history['val_loss']))]
fig , ax = plt.subplots(1,2)
train_acc = model.history.history['accuracy']
train_loss = model.history.history['loss']
val_acc = model.history.history['val_accuracy']
val_loss = model.history.history['val_loss']
fig.set_size_inches(20,10)

ax[0].plot(epochs , train_acc , 'g-' , label = 'Training Accuracy')
ax[0].plot(epochs , val_acc , 'r-' , label = 'Testing Accuracy')
ax[0].set_title('Training & Validation Accuracy')
ax[0].legend()
ax[0].set_xlabel("Epochs")
ax[0].set_ylabel("Accuracy")

ax[1].plot(epochs , train_loss , 'g-' , label = 'Training Loss')
ax[1].plot(epochs , val_loss , 'r-' , label = 'Testing Loss')
ax[1].set_title('Testing Accuracy & Loss')
ax[1].legend()
ax[1].set_xlabel("Epochs")
ax[1].set_ylabel("Loss")
plt.show()

In [ ]:
test_X = base_model.predict(test_X)
test_X = test_X.reshape(test_X.shape[0], shape)
test_X = test_X/train_X.max()

In [ ]:
print("Loss of the model is - " , model.evaluate(test_X,test_y)[0])
print("Accuracy of the model is - " , model.evaluate(test_X,test_y)[1]*100 , "%")